In [82]:
'''

This project is used data manipulation technique to study the terror case study.Motive of this project to find the suicide,
Number of killed people and Wounded people in each event.

'''

'\n\nThis project is used data manipulation technique to study the terror case study.Motive of this project to find the suicide,\nNumber of killed people and Wounded people in each event.\n\n'

In [63]:


import pandas as pd
import numpy as np
from pandas import DataFrame
import os
os.chdir('/Users/Desktop/Github/Data_Manipulation/Terror case study')

In [64]:
data=pd.read_csv('terror.csv',sep=',',header=0, encoding="latin")

print (data.head())
print (data.shape)
print (data.columns.tolist())


    eventid  iyear  imonth  iday approxdate  extended resolution  country  \
0  eventid1   1970       0     0        NaN         0        NaN       58   
1  eventid2   1970       0     0        NaN         0        NaN      130   
2  eventid3   1970       1     0        NaN         0        NaN      160   
3  eventid4   1970       1     0        NaN         0        NaN       78   
4  eventid5   1970       1     0        NaN         0        NaN      101   

          country_txt  region  ... addnotes scite1 scite2  scite3  dbsource  \
0  Dominican Republic       2  ...      NaN    NaN    NaN     NaN      PGIS   
1              Mexico       1  ...      NaN    NaN    NaN     NaN      PGIS   
2         Philippines       5  ...      NaN    NaN    NaN     NaN      PGIS   
3              Greece       8  ...      NaN    NaN    NaN     NaN      PGIS   
4               Japan       4  ...      NaN    NaN    NaN     NaN      PGIS   

   INT_LOG  INT_IDEO INT_MISC INT_ANY  related  
0        0   

/var/folders/mk/w5v6m8s178qb6zq684k2w9xh0000gn/T/ipykernel_2060/2539628933.py:1: DtypeWarning: Columns (4,61,62,66,116,117,123) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('terror.csv',sep=',',header=0, encoding="latin")


In [65]:

### How many attacks happened in India
data['country_txt'].unique().tolist()
data[data['country_txt']=="India"]
data[data['country_txt']=="India"].shape[0]


9940

In [66]:
## How many attacks happened in India and upto 3 people were killed
data[(data['country_txt']=='India')&(data['nkill']<=3)].shape[0]


8362

In [67]:
# This looks ugly
data.query("country_txt=='India' & nkill<=3").shape[0]


8362

In [68]:
# Extract the city and summary for attacks above
data.query("country_txt=='India' & nkill<=3")[['city','summary']].head(10)


,city,summary
1185,New Delhi,NaN
3780,New Delhi,NaN
5251,Bombay,NaN
7266,Imphal,NaN
8609,Unknown,NaN
8755,Unknown,NaN
8977,Amapur,NaN
9037,Raisikah,NaN
9040,Champhai,NaN
9070,Aizawl,NaN


In [69]:
# In a single terror incident in India, find out top 5 cities by number killed
data.query("country_txt=='India'").sort_values('nkill',ascending=False)['city'].head(5)


81000                Mumbai
96598               Jhargam
54339                Bombay
95860    Dantewada district
56837              Banabari
Name: city, dtype: object

In [70]:
# Little more detailed
data.query("country_txt=='India'").sort_values('nkill',ascending=False)[['city','nkill','iyear']].head(5)


,city,nkill,iyear
81000,Mumbai,188.0,2006
96598,Jhargam,115.0,2010
54339,Bombay,115.0,1992
95860,Dantewada district,82.0,2010
56837,Banabari,70.0,1994


In [71]:

# In a single terror incident in India, find out top 5 cities by number killed and wounded
data.query("country_txt=='India'").sort_values(['nkill','nwound'],ascending=[False,False])['city'].head(5)


81000                Mumbai
96598               Jhargam
54339                Bombay
95860    Dantewada district
56837              Banabari
Name: city, dtype: object

In [72]:
#### Adding new columns to the data
#Attacks which were successful and were suicide attacks
data['success_suicide']=data['suicide']+data['success']
data=data.assign(success_suicide_1=data['suicide']+data['success'])

data.query("success_suicide==2").shape[0]


4260

In [73]:

### Dropping columns
data.drop('success_suicide',axis=1) 
data.drop('success_suicide',axis=1,inplace=True) 


In [74]:

### Aggregations and manipulations using apply and map
#map: map a function to each element of a series object
data['nkill'].describe()
#Suppose we want to label all the incidents where the number killed was more than 5 as severe. This would involve applying a function on each element of the series, map helps in doing that
def get_label(x):
    if x>5:
        return 'Severe'
    else:
        return 'Not Severe'
data['nkill'].map(get_label)
#You can use lambda functions as well (if else follows a special form when used with lambdas )
data['nkill'].map(lambda x: "Severe" if x>5 else "Not Severe")

0         Not Severe
1         Not Severe
2         Not Severe
3         Not Severe
4         Not Severe
             ...    
156767    Not Severe
156768    Not Severe
156769    Not Severe
156770    Not Severe
156771    Not Severe
Name: nkill, Length: 156772, dtype: object

In [75]:
# We can use apply to use a function row wise or column wise
# Let's write a function to label an incident that was both successful and suicidal

def get_label(row):
    if row['success']==1 and row['suicide']==1:
        return 1
    else:
        return 0
data.apply(get_label,axis=1)
data.apply(get_label,axis=1).unique()

data.apply(lambda row: 1 if row['success']==1 and row['suicide']==1 else 0,axis=1)


0         0
1         0
2         0
3         0
4         0
         ..
156767    0
156768    0
156769    0
156770    0
156771    0
Length: 156772, dtype: int64

In [76]:

### Create a new  category representing if the incident occured in Afghanistan, Pakistan or India as one level of the category and all the other countries as another level

def get_label(row):
    if row['country_txt']=='India' or row['country_txt']=='Afghnistan' or row['country_txt']=='Pakistan':
        return 'Af-Pak-India'
    else:
        return 'ROW'
data.apply(get_label,axis=1)

data['Local']=data.apply(get_label,axis=1)


In [77]:
data.apply(lambda row: 'Af-Pak-India' if row['country_txt']=='India' or row['country_txt']=='Afghnistan' or row['country_txt']=='Pakistan' else  'ROW', axis =1 )

0         ROW
1         ROW
2         ROW
3         ROW
4         ROW
         ... 
156767    ROW
156768    ROW
156769    ROW
156770    ROW
156771    ROW
Length: 156772, dtype: object

In [78]:

## Number of incidents in Af-Pak-India vs ROW
data['Local'].value_counts()
data.groupby('Local').size()

Local
Af-Pak-India     22708
ROW             134064
dtype: int64

In [79]:


## Number of suicide attacks by Af-Pak-India vs ROW
data.groupby(['Local','suicide']).agg({'eventid':np.size})
data.groupby(['Local','suicide'],as_index=False).agg({'eventid':np.size})


,Local,suicide,eventid
0,Af-Pak-India,0,22228
1,Af-Pak-India,1,480
2,ROW,0,129773
3,ROW,1,4291


In [80]:

## Number of suicides attacks and average kills by Af-Pak-India vs ROW
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size])
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size]).reset_index()

,Local,suicide,mean,size
0,Af-Pak-India,0,1.539059,22228
1,Af-Pak-India,1,13.157620,480
2,ROW,0,2.192427,129773
3,ROW,1,10.159229,4291


In [81]:
#Rename columns
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size]).reset_index().rename(columns={'mean':'Average_Kills','size':'Number_Incidents'})

data.groupby(["Local",'suicide']).agg({'nkill':np.mean,'nwound':np.sum}).reset_index().rename(columns={'nwound':'Total_Wounded','nkill':'Number_Killed'})


,Local,suicide,Number_Killed,Total_Wounded
0,Af-Pak-India,0,1.539059,53341.00
1,Af-Pak-India,1,13.157620,11928.00
2,ROW,0,2.192427,292321.31
3,ROW,1,10.159229,82946.69
